# 실습2
## 무치차트 데이터 수집하기

In [1]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
#content > div.wArea.space > div.movieBox > ul > li:nth-child(1) > div > div.movie-title > h3 > a
#content > div.wArea.space > div.movieBox > ul > li:nth-child(1) > a > img

In [3]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os

movie_ranking = requests.get("https://www.moviechart.co.kr/rank/realtime/index/image")

image_dir = 'images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
    
pattern = r'[\\/:"*?<>|]'
movie_data = []

if movie_ranking.status_code == 200:
    print("영화 정보를 출력합니다.")
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')
    title_list = soup.select(".movieBox-list .movie-title a")
    image_list = soup.select(".movieBox-list .movieBox-item img")
    print(f"수집한 영화 수: {len(title_list)}")
    
    for title, image in zip(title_list, image_list):
        image_src = image.get('src')
        
         # ✅ 'source=' 뒤의 실제 이미지 URL만 추출
        source_match = re.search(r'source=(https?://.+)', image_src)
        if source_match:
            full_image_url = source_match.group(1)
        else:
            full_image_url = "https://www.moviechart.co.kr" + image_src  # fallback
        
        image_response = requests.get("https://www.moviechart.co.kr" + image_src)
        img = Image.open(BytesIO(image_response.content))
        image_filename = re.sub(pattern, '', title.text)
        img.save(os.path.join(image_dir, image_filename + ".png"))
        print(title.text)

        # 데이터 수집
        movie_data.append({'title': title.text, 'image_url': full_image_url})
else:
    print("페이지에 연결할 수 없습니다.")

import pandas as pd 
df = pd.DataFrame(movie_data)
df.to_csv("movie_list.csv", encoding="utf-8-sig", index=False)
print("✔ CSV 저장 완료: movie_list.csv")    

영화 정보를 출력합니다.
수집한 영화 수: 20
좀비딸
F1 더 무비
전지적 독자 시점
킹 오브 킹스
판타스틱 4: 새로운 출발
악마가 이사왔다
명탐정 코난: 척안의 잔상
발레리나
배드 가이즈 2
베베핀 극장판: 사라진 베베핀과 핑크퐁 대모험
쥬라기 월드: 새로운 시작
노이즈
콘스탄틴
극장판 도라에몽: 진구의 그림이야기
꼬마마법사 주니토니
고고다이노 극장판: 게코도마뱀의 꿈
슈퍼맨
미세리코르디아
강령: 귀신놀이
추적
✔ CSV 저장 완료: movie_list.csv


## db에 연결하기

In [4]:
import pymysql

# MySQL 연결
conn = pymysql.connect(
    host='localhost',      
    user='root',          
    password='1234',   
    db='web_data',           
    charset='utf8mb4'
)
cursor = conn.cursor()

# 테이블 생성
create_table_sql = '''
CREATE TABLE IF NOT EXISTS movies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    image_url TEXT
)
'''
cursor.execute(create_table_sql)
conn.commit()

In [5]:
import pandas as pd

# CSV 불러오기
df = pd.read_csv("movie_list.csv")

# INSERT 쿼리 준비
insert_sql = '''
INSERT INTO movies (title, image_url)
VALUES (%s, %s)
'''

# 데이터 삽입
for _, row in df.iterrows():
    cursor.execute(insert_sql, (row['title'], row['image_url']))

conn.commit()
print("✔ DB 저장 완료")

# 연결 종료
cursor.close()
conn.close()

✔ DB 저장 완료
